In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect,Integer

In [2]:
house = "./resources/housing_data.csv"
house_df = pd.read_csv(house)
house_lat_lng = "./resources/us_zip_lat_lng_data.csv"
house_lat_lng_df = pd.read_csv(house_lat_lng)
# house_lat_lng_df
# house_df
house_data = pd.merge(house_df, house_lat_lng_df, on = "zipcode")
house_data
# # Check to see if there are any rows with missing data
# house_data.count()

# Filter only the needed columns - we are only intersted in 2019 and 2020 data
# See what columns we hace available first

list(house_data.columns)

# Separate the data in two tables based on years (2019, 2020)
house_data_2019 = house_data[['zipcode', 'lat', 'lng','state', 'city',
                             '1/31/2019',
                             '2/28/2019',
                             '3/31/2019',
                             '4/30/2019',
                             '5/31/2019',
                             '6/30/2019',
                             '7/31/2019',
                             '8/31/2019',
                             '9/30/2019',
                             '10/31/2019',
                             '11/30/2019',
                             '12/31/2019']]
house_data_2020 = house_data[['zipcode', 'lat', 'lng','state', 'city',
                             '1/31/2020',
                             '2/29/2020',
                             '3/31/2020',
                             '4/30/2020',
                             '5/31/2020',
                             '6/30/2020',
                             '7/31/2020',
                             '8/31/2020',
                             '9/30/2020',
                             '10/31/2020',
                             '11/30/2020',
                             '12/31/2020']]

In [3]:
# Remove the rows with missing data
house_data_2019 = house_data_2019.dropna(how = "any")
house_data_2020 = house_data_2020.dropna(how = "any")
house_data_2019

,zipcode,lat,lng,state,city,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019
0,10025,40.798601,-73.966622,NY,New York,1101403.0,1106902.0,1118061.0,1109151.0,1104925.0,1093784.0,1073845.0,1042718.0,1040381.0,1056542.0,1091406.0,1103597.0
1,60657,41.940293,-87.646857,IL,Chicago,492609.0,490791.0,488931.0,488836.0,487848.0,486797.0,486557.0,485590.0,484797.0,483322.0,482671.0,482194.0
2,10023,40.775921,-73.982607,NY,New York,1478226.0,1439971.0,1437751.0,1427383.0,1424757.0,1418924.0,1423161.0,1416499.0,1385727.0,1372936.0,1387600.0,1426845.0
3,77494,29.740677,-95.829652,TX,Katy,340918.0,340452.0,340308.0,339298.0,338608.0,338047.0,338293.0,338411.0,339205.0,339613.0,339952.0,339667.0
4,60614,41.922714,-87.649577,IL,Chicago,637963.0,635863.0,633659.0,634383.0,632924.0,631094.0,630080.0,629085.0,628363.0,627307.0,626560.0,625729.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29933,89405,40.480566,-119.424671,NV,Gerlach,303561.0,300503.0,293876.0,290859.0,283394.0,278953.0,274569.0,268279.0,264584.0,261091.0,260871.0,259621.0
29934,86343,34.176983,-112.299986,AZ,Crown King,165028.0,168741.0,168319.0,168224.0,165392.0,166651.0,167534.0,167854.0,167001.0,165520.0,165662.0,165661.0
29935,47865,39.020999,-87.391428,IN,Carlisle,35360.0,35648.0,35898.0,36428.0,36836.0,37216.0,37349.0,38224.0,38852.0,39118.0,38431.0,37633.0
29936,20052,38.900126,-77.046981,DC,Washington,1286415.0,1288474.0,1300984.0,1306419.0,1311746.0,1310135.0,1307794.0,1306515.0,1312888.0,1323328.0,1341949.0,1350843.0


In [4]:
# get a columns list of years only
column_list_2019 = list(house_data_2019.columns)
year_2019 = [i for i in column_list_2019 if i not in ["zipcode","lat","lng","state","city"]]
column_list_2020 = list(house_data_2020.columns)
year_2020 = [i for i in column_list_2020 if i not in ["zipcode","lat","lng","state","city"]]


In [5]:
# take the average for all month together for a year
avg_2019 = house_data_2019[year_2019].mean(axis=1)
pd.options.display.float_format = '{:,.2f}'.format
house_data_2019["avg2019"] = avg_2019
avg_2020 = house_data_2020[year_2020].mean(axis=1)
house_data_2020["avg2020"] = avg_2020

In [12]:
# do the final cleaning
house_data_2019 = house_data_2019[['zipcode', 'lat', 'lng','state', 'city','avg2019']]
house_data_2020 = house_data_2020[['zipcode', 'lat', 'lng','state', 'city','avg2020']] 

# merge this two table in one
final_data = house_data_2019.merge(house_data_2020)
final_data.to_csv("./resources/final_data.csv", index=False)


In [14]:
# now we have the final data ready to be pushed into a data base
!pip install csv_to_sqlite
import csv_to_sqlite
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="windows-1250") 
input_files = ["./resources/final_data.csv"]
csv_to_sqlite.write_csv(input_files, "./resources/sqlite/final_data.sqlite", options)


Written 29377 rows into 1 tables in 0.486 seconds


29377

In [17]:
# Create an engine that can talk to the database
engine = create_engine("sqlite:///./resources/sqlite/final_data.sqlite")

# Explore Database
inspector = inspect(engine)
print("Working with the following tables:  ")
table_name = inspector.get_table_names()
inspector.get_columns(table_name[0])


Working with the following tables:  


[{'name': 'zipcode',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'lat',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'lng',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'state',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'city',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'avg2019',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'avg2020',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]